<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_planetary_ai_full_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_planetary_ai_full.py

Complete pipeline for PlanetaryAI with uncertainty quantification:
1. Synthetic “planetary” dataset of 6 inputs → 3 outputs
2. Standardization, noise injection
3. MLP with residual head & dropout
4. Training loop with gradient clipping, LR scheduler, checkpointing
5. Validation and loss reporting
6. MC-Dropout inference for uncertainty quantification
7. Heatmap of uncertainty over ICL vs. PRA
8. Scatter plots of true vs. predicted
"""

import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split

# ------------------------------------------------------------------------------
# 1. Synthetic Planetary Dataset
# ------------------------------------------------------------------------------
class PlanetaryDataset(Dataset):
    def __init__(self, n_samples=5000, seed=42):
        torch.manual_seed(seed)
        data = torch.rand(n_samples, 6)  # CNS, CER, ICL, PRA, ECI, CDF
        CNS, CER, ICL, PRA, ECI, CDF = data.t()
        ε = 1e-6

        coherence  = (CNS * CER) / (ICL + ε)
        efficiency = PRA / (ICL + 1.0)
        retention  = CDF * torch.exp(-ECI)

        targets = torch.stack([coherence, efficiency, retention], dim=1)
        targets += 0.01 * targets.std(0) * torch.randn_like(targets)

        self.X_mean, self.X_std = data.mean(0), data.std(0) + ε
        self.Y_mean, self.Y_std = targets.mean(0), targets.std(0) + ε

        self.X = ((data - self.X_mean) / self.X_std).float()
        self.Y = ((targets - self.Y_mean) / self.Y_std).float()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# ------------------------------------------------------------------------------
# 2. PlanetaryAI Model with Residuals & Dropout
# ------------------------------------------------------------------------------
class PlanetaryAI(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=32, output_dim=3, p_drop=0.1):
        super().__init__()
        self.fc1      = nn.Linear(input_dim, hidden_dim)
        self.res_head = nn.Linear(input_dim, hidden_dim)
        self.fc2      = nn.Linear(hidden_dim, hidden_dim)
        self.fc3      = nn.Linear(hidden_dim, output_dim)
        self.relu     = nn.ReLU()
        self.drop     = nn.Dropout(p_drop)

    def forward(self, x):
        h1 = self.relu(self.fc1(x))
        r  = self.res_head(x)
        h2 = self.drop(self.relu(self.fc2(h1 + r)))
        out = self.fc3(h2)
        return out

# ------------------------------------------------------------------------------
# 3. Training & Validation
# ------------------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0.0
    criterion = nn.MSELoss()
    for Xb, Yb in loader:
        Xb, Yb = Xb.to(device), Yb.to(device)
        pred = model(Xb)
        loss = criterion(pred, Yb)
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item() * Xb.size(0)
    return total_loss / len(loader.dataset)

def validate(model, loader, device):
    model.eval()
    total_loss = 0.0
    criterion = nn.MSELoss()
    with torch.no_grad():
        for Xb, Yb in loader:
            Xb, Yb = Xb.to(device), Yb.to(device)
            pred = model(Xb)
            total_loss += criterion(pred, Yb).item() * Xb.size(0)
    return total_loss / len(loader.dataset)

# ------------------------------------------------------------------------------
# 4. MC-Dropout Uncertainty Quantification
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, X, T=50):
    model.train()  # keep dropout on
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(X))
    stacked = torch.stack(preds, dim=0)
    return stacked.mean(0), stacked.std(0)

def plot_uncertainty_heatmap(model, stats, device):
    # Vary ICL vs. PRA; keep others at mean
    grid_size = 100
    ICL = np.linspace(0,1,grid_size, dtype=np.float32)
    PRA = np.linspace(0,1,grid_size, dtype=np.float32)
    IC, PR = np.meshgrid(ICL, PRA)
    pts = grid_size**2

    Xg = torch.zeros((pts,6), device=device)
    # CNS, CER, ECI, CDF = mean
    for i in [0,1,4,5]:
        Xg[:,i] = stats['X_mean'][i]
    Xg[:,2] = torch.from_numpy(IC.ravel()).to(device)
    Xg[:,3] = torch.from_numpy(PR.ravel()).to(device)

    Xn = (Xg - stats['X_mean']) / stats['X_std']
    _, std = mc_dropout_predict(model, Xn, T=100)
    U = std[:,0].cpu().reshape(IC.shape)

    plt.figure(figsize=(5,4))
    plt.pcolormesh(IC, PR, U, cmap='magma', shading='auto')
    plt.colorbar(label='Std(Coherence)')
    plt.xlabel('Interplanetary Latency (ICL)')
    plt.ylabel('Resource Availability (PRA)')
    plt.title('Uncertainty Heatmap: Coherence')
    plt.tight_layout()
    plt.show()

# ------------------------------------------------------------------------------
# 5. Main
# ------------------------------------------------------------------------------
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Data
    ds     = PlanetaryDataset(n_samples=5000, seed=42)
    val_n  = int(0.2 * len(ds))
    tr_ds, va_ds = random_split(ds, [len(ds)-val_n, val_n])
    tr_ld  = DataLoader(tr_ds, batch_size=128, shuffle=True)
    va_ld  = DataLoader(va_ds, batch_size=256, shuffle=False)

    # Stats for denorm/UQ
    stats = {
        'X_mean': ds.X_mean.to(device),
        'X_std' : ds.X_std.to(device),
        'Y_mean': ds.Y_mean.to(device),
        'Y_std' : ds.Y_std.to(device),
    }

    # Model, optimizer, scheduler
    model     = PlanetaryAI().to(device)
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
    sched     = optim.lr_scheduler.ReduceLROnPlateau(
                    optimizer, mode='min', factor=0.5, patience=5
                )

    os.makedirs("checkpoints", exist_ok=True)
    best_val = float('inf')

    for ep in range(1, 51):
        tr_loss = train_one_epoch(model, tr_ld, optimizer, device)
        va_loss = validate(model, va_ld, device)
        sched.step(va_loss)
        print(f"Epoch {ep:02d} | Train Loss: {tr_loss:.4f} | Val Loss: {va_loss:.4f}")

        if va_loss < best_val - 1e-5:
            best_val = va_loss
            torch.save(model.state_dict(), "checkpoints/planetary_best.pth")

    # Load best checkpoint
    model.load_state_dict(torch.load("checkpoints/planetary_best.pth", map_location=device))
    print("Best model loaded.")

    # Scatter plots of true vs. predicted
    X_all, Y_all = ds.X.to(device), ds.Y.to(device)
    with torch.no_grad():
        Y_pred = model(X_all)

    Y_true = (Y_all * ds.Y_std.to(device)) + ds.Y_mean.to(device)
    Y_est  = (Y_pred * ds.Y_std.to(device)) + ds.Y_mean.to(device)

    for i, name in enumerate(["Coherence", "Efficiency", "Retention"]):
        plt.figure(figsize=(4,4))
        plt.scatter(Y_true[:,i].cpu(), Y_est[:,i].cpu(), s=5, alpha=0.6)
        m, M = Y_true[:,i].min(), Y_true[:,i].max()
        plt.plot([m, M], [m, M], 'r--')
        plt.title(name)
        plt.xlabel("True")
        plt.ylabel("Predicted")
        plt.tight_layout()
        plt.show()

    # Uncertainty heatmap
    plot_uncertainty_heatmap(model, stats, device)


if __name__ == "__main__":
    main()